## ch09. 추천시스템

### 01) 추천시스템의 개요와 배경

- 하나의 콘텐츠를 선택했을 때 선택된 콘텐츠와 연관된 추천 콘텐츠가 얼마나 사용자의 관심을 끌고 개인에게 맞춘 콘텐츠를 추천했는지는 그 사이트의 평판을 좌우하는 매우 중요한 요소임.

- 추천시스템의 진정한 묘미는 사용자 자신도 몰랐던취향을 시스템이 발견하고 그에 맞틑 콘텐츠를 추천해주는 것.
=> 사용자는 해당 사이트를 더 강하게 신뢰하게 되어 더 많은 추천 콘텐츠를 선택하게 됨.



#### 추천시스템을 구성하는 데이터

    - 사용자가 어떤 상품을 구매했는가?
    - 사용자가 어떤 상품을 둘러보거나 장바구니에 넣었는가?
    - 사용자가 평가한 영화 평점은? 제품 평가는?
    - 사용자가 스스로 작성한 자신의 취향은?
    - 사용자가 무엇을 클릭했는가?
    

### 추천시스템의 유형
1. **콘텐츠 기반 필터링(Content based filtering)** : 사용자가 특정한 아이템을 매우 선호하는 경우, 그 아이템과 비슷한 콘텐츠를 가진 다른 아이템을 추천하는 방식
    
    ex) 사용자가 특정 영화에 높은 평점을 줬다면 그 영화의 장르, 출연 배우, 감독, 영화 키워드 등의 콘텐츠와 유사한 다른 영화를 추천해주는 방식 
    
    
2. **협업 필터링(Collaborative filtering)** : 사용자가 아이템에 메긴 평점 정보나 상품 구매 이력과 같은 사용자 행동 양식만을 기반으로 추천을 수행하는 것, 사용자가 평가한 다른 아이템을 기반으로 사용자가 평가하지 않은 아이템의 예측 평가를 도출하는 방식
    - 행렬 데이터에만 의지해 추천 수행 => 행은 개별 사용자, 열은 개별 아이템
    - **최근접 이웃 협업 필터링** : 
        - 사용자 기반 필터링 : 당신과 비슷한 고객들이 다음 상품도 구매했다.
![r6](https://user-images.githubusercontent.com/84848848/151555216-fe27dfa9-13fe-4ed8-a5c6-a29dbe3d6119.jpg)

        - 아이템 기반 필터링 :  이 상품을 선택한 다른 고객들은 다음 상품도 구매했다.
![r7](https://user-images.githubusercontent.com/84848848/151555225-a5f270b4-02b8-4c91-925d-c2d61f772f7c.jpg)

        - 일반적으로 아이템 기반이 정확도가 높음 => 사람들의 취향이 비슷하다고 판단하기 어려움 => **최근접 이웃 필터링은 대부분 아이템 기반 필터링 적용**
        - 유사도 측정은 코사인 유사도 이용        
        
    - **잠재요인 협업 필터링** : 사용자-아이템 평점 행렬 데이터만을 이용해 말 그대로 '잠재요인'을 끄집어 내는 것. '잠재 요인'이 어떤 것인지는 명확히 정의할 수 없음
    ![r8](https://user-images.githubusercontent.com/84848848/151707330-38bbeb39-4025-447e-8a4a-2db0c0ad1663.png)


### 확률적 경사 하강법을 이용한 행렬 분해

: P와 Q행렬로 계산된 예측 R 행렬 값이 실제 R 행렬 값과 가장 최소의 오류를 가질 수 있도록 반복적인 비용 함수 최적화를 통해 P와 Q를 유추해내는 것

    1. 임의의 값을 가진 P,Q 설정
    2. P와 Q.T 값을 곱해 예측 행렬 R 계산하고 실제 행렬과의 오류값 계산
    3. 오류값 최소화하도록 P,Q 업데이트
    4. 만족할만한 오류값 가질때까지 근사화
![s5](https://user-images.githubusercontent.com/84848848/151778474-7e8ac09f-70ce-4b19-9e36-6229b2fa0e8a.png)
![s6](https://user-images.githubusercontent.com/84848848/151778503-71df67f2-2010-4d6c-8177-6e24fbb0b167.png)
    

In [3]:
import numpy as np

# 원본 행렬 R 생성, 잠재요인 차원 k=3
R = np.array([[4, np.NaN, np.NaN, 2, np.NaN ],
              [np.NaN, 5, np.NaN, 3, 1 ],
              [np.NaN, np.NaN, 3, 4, 4 ],
              [5, 2, 1, 2, np.NaN ]])
num_users, num_items = R.shape
K=3

# 임의의 P,Q 생성
np.random.seed(1)
P = np.random.normal(scale=1./K, size=(num_users, K))
Q = np.random.normal(scale=1./K, size=(num_items, K))

In [57]:
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, not_nan_index):
    error = 0
    
    # 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)
    
    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
      
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

In [59]:
non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]
non_zeros

[(0, 0, 4.0),
 (0, 3, 2.0),
 (1, 1, 5.0),
 (1, 3, 3.0),
 (1, 4, 1.0),
 (2, 2, 3.0),
 (2, 3, 4.0),
 (2, 4, 4.0),
 (3, 0, 5.0),
 (3, 1, 2.0),
 (3, 2, 1.0),
 (3, 3, 2.0)]

In [61]:
# 실제 R 행렬에서 Null이 아닌 index
not_nan_index = np.where(np.isnan(R) == False)

# 반복수, 학습률, L2 규제
steps=1000
learning_rate=0.01
r_lambda=0.01

for step in range(steps):  # 1000회 업데이트
    for i, j, r in non_zeros:
        
        # 실제 값과 예측 값의 차이인 오류 값 구함
        eij = r - np.dot(P[i, :], Q[j, :].T)
        
        # Regularization을 반영한 SGD(확률적 경사하강법) 업데이트 공식 적용
        P[i,:] = P[i,:] + learning_rate * ( eij * Q[j,:] - r_lambda*P[i,:] )
        Q[j,:] = Q[j,:] + learning_rate * ( eij * P[i,:] - r_lambda*Q[j,:] )

    rmse = get_rmse(R, P, Q, non_zeros)
    if (step % 50) == 0 :
        print("### iteration step : ", step," rmse : ", rmse)

### iteration step :  0  rmse :  0.013604268005178563
### iteration step :  50  rmse :  0.013601676836046014
### iteration step :  100  rmse :  0.01359911465760444
### iteration step :  150  rmse :  0.013596581089665811
### iteration step :  200  rmse :  0.013594075757850934
### iteration step :  250  rmse :  0.013591598293481553
### iteration step :  300  rmse :  0.013589148333483908
### iteration step :  350  rmse :  0.01358672552028806
### iteration step :  400  rmse :  0.013584329501727506
### iteration step :  450  rmse :  0.013581959930950523
### iteration step :  500  rmse :  0.013579616466320582
### iteration step :  550  rmse :  0.013577298771331743
### iteration step :  600  rmse :  0.013575006514514472
### iteration step :  650  rmse :  0.013572739369347398
### iteration step :  700  rmse :  0.0135704970141766
### iteration step :  750  rmse :  0.013568279132127655
### iteration step :  800  rmse :  0.01356608541102451
### iteration step :  850  rmse :  0.01356391554330565
#

In [63]:
pred_matrix = np.dot(P, Q.T)
print('예측 행렬:\n', np.round(pred_matrix, 3))

print("-"*35)

print('실제 행렬:')
print(R)

예측 행렬:
 [[3.991 1.991 1.104 1.997 1.557]
 [3.982 4.978 1.148 2.987 1.005]
 [4.848 2.663 2.988 3.981 3.984]
 [4.974 2.001 1.003 2.001 1.553]]
-----------------------------------
실제 행렬:
[[ 4. nan nan  2. nan]
 [nan  5. nan  3.  1.]
 [nan nan  3.  4.  4.]
 [ 5.  2.  1.  2. nan]]


### 02) 콘텐츠 기반 필터링 실습

In [6]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')
movies =pd.read_csv('tmdb_5000_movies.csv')
movies_df = movies[['id','title', 'genres', 'vote_average', 'vote_count',
                 'popularity', 'keywords', 'overview']]
pd.set_option('max_colwidth', 100)
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


In [7]:
from ast import literal_eval
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

In [8]:
movies_df['genres'] = movies_df['genres'].apply(lambda x : [ y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [ y['name'] for y in x])
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


### 장르 콘텐츠 유사도 측정

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환. 
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


In [12]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:1])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]]


In [11]:
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
print(genre_sim_sorted_ind[:1])

[[   0 3494  813 ... 3038 3037 2401]]


### 장르 콘텐츠 필터링을 이용한 영화 추천

In [13]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    
    # 인자로 입력된 movies_df DataFrame에서 'title' 컬럼이 입력된 title_name 값인 DataFrame추출
    title_movie = df[df['title'] == title_name]
    
    # title_named을 가진 DataFrame의 index 객체를 ndarray로 반환하고 
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n 개의 index 추출
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    # 추출된 top_n index들 출력. top_n index는 2차원 데이터 임. 
    #dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

In [16]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather',10)
similar_movies[['title','vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


In [17]:
movies_df[['title','vote_average','vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


In [19]:
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)
print('C: ',round(C,3),'m: ',round(m,3))

C:  6.092 m:  370.2


In [20]:
percentile = 0.6
m = movies_df['vote_count'].quantile(percentile)
C = movies_df['vote_average'].mean()
def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ( (v/(v+m)) * R ) + ( (m/(m+v)) * C )

movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1) 

movies_df[['title','vote_average','weighted_vote','vote_count']].sort_values('weighted_vote',
                                                                          ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


In [21]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    # top_n의 2배에 해당하는 쟝르 유사성이 높은 index 추출 
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1)
# 기준 영화 index는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    # top_n의 2배에 해당하는 후보군에서 weighted_vote 높은 순으로 top_n 만큼 추출 
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather',10)
similar_movies[['title', 'vote_average', 'weighted_vote']]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427


### 02) 아이템 기반 최근접 이웃 협업 필터링 실습

In [24]:
movies = pd.read_csv("./ml-latest-small/movies.csv")
ratings = pd.read_csv("./ml-latest-small/ratings.csv")
print(movies.shape)
print(ratings.shape)

(9742, 3)
(100836, 4)


In [25]:
ratings_matrix = ratings.pivot_table("rating", "userId", "movieId")
ratings_matrix.head(3)

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# ratings 데이터와 movies 데이터 결합
rating_movies = pd.merge(ratings, movies, on="movieId")

# 사용자-아이템 평점 행렬 생성
ratings_matrix = rating_movies.pivot_table("rating", "userId", "title")

# NaN값은 0으로 변환
ratings_matrix.fillna(0, inplace=True)

# 아이템-사용자 평점 행렬로 전치
ratings_matrix_T = ratings_matrix.T
ratings_matrix_T.head(3)

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
from sklearn.metrics.pairwise import cosine_similarity

# 아이템 유사도 행렬
item_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)

# 데이터 프레임 형태로 저장
item_sim_df = pd.DataFrame(item_sim, index=ratings_matrix.columns, columns=ratings_matrix.columns)

print(item_sim_df.shape)
item_sim_df.head(3)

(9719, 9719)


title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),1.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.141653,0.0,...,0.0,0.342055,0.543305,0.707107,0.0,0.0,0.139431,0.327327,0.0,0.0
'Hellboy': The Seeds of Creation (2004),0.0,1.000000,0.707107,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Round Midnight (1986),0.0,0.707107,1.000000,0.0,0.0,0.0,0.176777,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0


In [29]:
item_sim_df["Godfather, The (1972)"].sort_values(ascending=False)[:6]

title
Godfather, The (1972)                        1.000000
Godfather: Part II, The (1974)               0.821773
Goodfellas (1990)                            0.664841
One Flew Over the Cuckoo's Nest (1975)       0.620536
Star Wars: Episode IV - A New Hope (1977)    0.595317
Fargo (1996)                                 0.588614
Name: Godfather, The (1972), dtype: float64

In [31]:
item_sim_df["Inception (2010)"].sort_values(ascending=False)[1:6]

title
Dark Knight, The (2008)          0.727263
Inglourious Basterds (2009)      0.646103
Shutter Island (2010)            0.617736
Dark Knight Rises, The (2012)    0.617504
Fight Club (1999)                0.615417
Name: Inception (2010), dtype: float64

In [32]:
def predict_rating(ratings_arr, item_sim_arr):
    ratings_pred = ratings_arr.dot(item_sim_arr) / np.array([np.abs(item_sim_arr).sum(axis=1)])
    
    return ratings_pred

In [33]:
ratings_pred = predict_rating(ratings_matrix.values , item_sim_df.values)

ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index= ratings_matrix.index,
                                   columns = ratings_matrix.columns)
ratings_pred_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.070345,0.577855,0.321696,0.227055,0.206958,0.194615,0.249883,0.102542,0.157084,0.178197,...,0.113608,0.181738,0.133962,0.128574,0.006179,0.212070,0.192921,0.136024,0.292955,0.720347
2,0.018260,0.042744,0.018861,0.000000,0.000000,0.035995,0.013413,0.002314,0.032213,0.014863,...,0.015640,0.020855,0.020119,0.015745,0.049983,0.014876,0.021616,0.024528,0.017563,0.000000
3,0.011884,0.030279,0.064437,0.003762,0.003749,0.002722,0.014625,0.002085,0.005666,0.006272,...,0.006923,0.011665,0.011800,0.012225,0.000000,0.008194,0.007017,0.009229,0.010420,0.084501


In [34]:
from sklearn.metrics import mean_squared_error

# 성능 평가는 MSE를 사용
def get_mse(pred, actual):
    # 평점이 있는 실제 영화만 추출 (1차원 배열로 변환)
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    
    return mean_squared_error(pred, actual)

print('아이템 기반 모든 최근접 이웃 mse: ', get_mse(ratings_pred, ratings_matrix.values))

아이템 기반 모든 최근접 이웃 mse:  9.895354759094706


In [41]:
def predict_rating_topsim(ratings_arr, item_sim_arr, n=20):
    pred = np.zeros(ratings_arr.shape)

    for col in range(ratings_arr.shape[1]):
        # 아이템의 유사도 상위 N개 아이템
        top_n_items = [np.argsort(item_sim_arr[:, col])[:-1-n:-1]]
        
        # 개인화된 예측 평점을 계산: 반복당 특정 아이템의 예측 평점(사용자 전체)
        for row in range(ratings_arr.shape[0]):
            # 예측 평점
            pred[row, col] = item_sim_arr[col,:][top_n_items].dot(ratings_arr[row, :][top_n_items].T)
            pred[row, col] /= np.sum( np.abs(item_sim_arr[col,:][top_n_items]))
            
    return pred

In [43]:
ratings_pred = predict_rating_topsim(ratings_matrix.values, item_sim_df.values,n=20)
print('아이템 기반 최근접 Top20 이웃 mse: ', get_mse(ratings_pred,ratings_matrix.values))

아이템 기반 최근접 Top20 이웃 mse:  3.694973960779922


In [44]:
ratings_pred_matrix = pd.DataFrame(data=ratings_pred,index=ratings_matrix.index,columns=ratings_matrix.columns)

In [49]:
user_rating_id=ratings_matrix.loc[9,:]
user_rating_id[user_rating_id>0].sort_values(ascending=False)[:10]

title
Adaptation (2002)                                                                 5.0
Citizen Kane (1941)                                                               5.0
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)    5.0
Producers, The (1968)                                                             5.0
Lord of the Rings: The Two Towers, The (2002)                                     5.0
Lord of the Rings: The Fellowship of the Ring, The (2001)                         5.0
Back to the Future (1985)                                                         5.0
Austin Powers in Goldmember (2002)                                                5.0
Minority Report (2002)                                                            4.0
Witness (1985)                                                                    4.0
Name: 9, dtype: float64

In [50]:
def get_unseen_movies(ratings_matrix, userId):
    user_rating = ratings_matrix.loc[userId,:]
    already_seen = user_rating[user_rating>0].index.tolist()
    movie_list = ratings_matrix.columns.tolist()
    unseen_list = [movie for movie in movie_list if movie not in already_seen]
    return unseen_list

In [51]:
def recomm_movie_by_userId(pred_df,userId,unseen_list,top_n=10):
    recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_movies

unseen_list = get_unseen_movies(ratings_matrix,9)
recomm_movies = recomm_movie_by_userId(ratings_pred_matrix,9,unseen_list,top_n=10)
recomm_movies = pd.DataFrame(data=recomm_movies.values, index=recomm_movies.index,columns=['pred_score'])
recomm_movies

,pred_score
title,
Shrek (2001),0.866202
Spider-Man (2002),0.857854
"Last Samurai, The (2003)",0.817473
Indiana Jones and the Temple of Doom (1984),0.816626
"Matrix Reloaded, The (2003)",0.800990
Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001),0.765159
Gladiator (2000),0.740956
"Matrix, The (1999)",0.732693
Pirates of the Caribbean: The Curse of the Black Pearl (2003),0.689591


### 03) 행렬 분해를 이용한 잠재 요인 협업 필터링 실습

In [67]:
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    
    # 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)
    
    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
      
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

In [71]:
def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda=0.01):
    num_users, num_items = R.shape

    np.random.seed(1)
    P = np.random.normal(scale=1.0/K, size=(num_users, K))
    Q = np.random.normal(scale=1.0/K, size=(num_items, K))
    
    prev_rmse = 1000
    break_count = 0

    # R>0인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장한다.
    non_zeros = [ (i, j, R[i, j]) for i in range(num_users) for j in range(num_items) if R[i, j] > 0 ]

    # SGD 기법으로 P, Q 매트릭스를 업데이트 함
    for step in range(steps):
        for i, j, r in non_zeros:
            # 잔차 구함
            eij = r - np.dot(P[i, :], Q[j, :].T)

            # Regulation을 반영한 SGD 업데이터 적용
            P[i, :] = P[i, :] + learning_rate*(eij * Q[j, :] - r_lambda*P[i, :])
            Q[j, :] = Q[j, :] + learning_rate*(eij * P[i, :] - r_lambda*Q[j, :])

        rmse = get_rmse(R, P, Q, non_zeros)
        if step % 10 == 0:
            print("iter step: ", step,"rmse: ", rmse)
            
    return P, Q

In [53]:
movies = pd.read_csv("./ml-latest-small/movies.csv")
ratings = pd.read_csv("./ml-latest-small/ratings.csv")
ratings_matrix = ratings.pivot_table("rating", "userId", "movieId")

In [55]:
rating_movies = pd.merge(ratings,movies,on='movieId')
ratings_matrix = rating_movies.pivot_table('rating',index='userId',columns='title')

In [72]:
P, Q = matrix_factorization(ratings_matrix.values, K=50, steps=200, learning_rate=0.01, r_lambda=0.01)
pred_matrix = np.dot(P,Q.T)

iter step:  0 rmse:  2.9023619751336867
iter step:  10 rmse:  0.7335768591017927
iter step:  20 rmse:  0.5115539026853442
iter step:  30 rmse:  0.37261628282537446
iter step:  40 rmse:  0.2960818299181014
iter step:  50 rmse:  0.2520353192341642
iter step:  60 rmse:  0.22487503275269854
iter step:  70 rmse:  0.2068545530233154
iter step:  80 rmse:  0.19413418783028685
iter step:  90 rmse:  0.18470082002720403
iter step:  100 rmse:  0.17742927527209104
iter step:  110 rmse:  0.17165226964707492
iter step:  120 rmse:  0.16695181946871723
iter step:  130 rmse:  0.1630529219199754
iter step:  140 rmse:  0.15976691929679646
iter step:  150 rmse:  0.1569598699945732
iter step:  160 rmse:  0.1545339818671543
iter step:  170 rmse:  0.15241618551077643
iter step:  180 rmse:  0.1505508073962831
iter step:  190 rmse:  0.14889470913232092


In [73]:
ratings_pred_matrix = pd.DataFrame(data=pred_matrix,index=ratings_matrix.index,columns=ratings_matrix.columns)
ratings_pred_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,3.055084,4.092018,3.564130,4.502167,3.981215,1.271694,3.603274,2.333266,5.091749,3.972454,...,1.402608,4.208382,3.705957,2.720514,2.787331,3.475076,3.253458,2.161087,4.010495,0.859474
2,3.170119,3.657992,3.308707,4.166521,4.311890,1.275469,4.237972,1.900366,3.392859,3.647421,...,0.973811,3.528264,3.361532,2.672535,2.404456,4.232789,2.911602,1.634576,4.135735,0.725684
3,2.307073,1.658853,1.443538,2.208859,2.229486,0.780760,1.997043,0.924908,2.970700,2.551446,...,0.520354,1.709494,2.281596,1.782833,1.635173,1.323276,2.887580,1.042618,2.293890,0.396941


In [74]:
# 사용자가 관람하지 않은 영화명
unseen_list = get_unseen_movies(ratings_matrix,9)

# 잠재 요인 협업 필터링으로 영화 추천
recomm_movies = recomm_movie_by_userId(ratings_pred_matrix,9,unseen_list,top_n=10)

# 평점 데이터를 DataFrame으로 생성
recomm_movies = pd.DataFrame(data=recomm_movies.values,index=recomm_movies.index,columns=['pred_score'])
recomm_movies

,pred_score
title,
Rear Window (1954),5.704612
"South Park: Bigger, Longer and Uncut (1999)",5.451100
Rounders (1998),5.298393
Blade Runner (1982),5.244951
Roger & Me (1989),5.191962
Gattaca (1997),5.183179
Ben-Hur (1959),5.130463
Rosencrantz and Guildenstern Are Dead (1990),5.087375
"Big Lebowski, The (1998)",5.038690


### 04) 파이썬 추천 시스템 패키지 - Surprise

In [3]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [4]:
data = Dataset.load_builtin('ml-100k')

# surprise의 train_test_split() 사용
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\qkral/.surprise_data/ml-100k


In [5]:
# SVD를 이용한 잠재 요인 협업 필터링
algo = SVD()
algo.fit(trainset)

In [6]:
predictions = algo.test( testset )

print('prediction type :',type(predictions), ' size:',len(predictions))
print('prediction 결과의 최초 5개 추출')

predictions[:5]

prediction type : <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.8455672757279253, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.764950701986391, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=3.939811808245712, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.4507207272948066, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.23599529875921, details={'was_impossible': False})]

In [7]:
# 속성 확인
[ (pred.uid, pred.iid, pred.est, pred.details) for pred in predictions[:3] ]

[('120', '282', 3.8455672757279253, {'was_impossible': False}),
 ('882', '291', 3.764950701986391, {'was_impossible': False}),
 ('535', '507', 3.939811808245712, {'was_impossible': False})]

In [8]:
# 사용자 아이디(uid), 아이템 아이디(iid)는 문자열로 입력
uid = str(196)
iid = str(302)

# 추천 예측 평점 (.predict)
pred = algo.predict(uid, iid)
pred

Prediction(uid='196', iid='302', r_ui=None, est=4.20713241370944, details={'was_impossible': False})

In [9]:
accuracy.rmse(predictions)

RMSE: 0.9478


0.9478053088839774

### 영화 데이터셋

In [11]:
import pandas as pd
ratings = pd.read_csv('./ml-latest-small/ratings.csv')
ratings.to_csv('./ml-latest-small/ratings_noh.csv', index=False, header=False)

In [12]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data = Dataset.load_from_file('./ml-latest-small/ratings_noh.csv', reader=reader)

In [13]:
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

# SVD를 이용한 잠재 요인 협업 필터링 (잠재 요인 크기 = 50)
algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)

# 추천 예측 평점 (.test)
predictions = algo.test( testset )

# 성능 평가
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

#### pandas에서 surprise로 데이터 로드

In [14]:
# 데이터 불러오기 (데이터 프레임)
ratings = pd.read_csv('./ml-latest-small/ratings.csv') 

# Reader 객체 생성
reader = Reader(rating_scale=(0.5, 5.0))

# 사용자 아이디, 아이템 아이디, 평점 순서 (원래는 timestamp도 있으나 제외)
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# surprise의 train_test_split() 사용
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

# SVD를 이용한 잠재 요인 협업 필터링 (잠재 요인 크기 = 50)
algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)

# 추천 예측 평점 (.test)
predictions = algo.test( testset )

# 성능 평가
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

#### 하이퍼 파라미터 튜닝&교차검증

In [15]:
from surprise.model_selection import cross_validate 

# 데이터 불러오기 (데이터 프레임)
ratings = pd.read_csv('./ml-latest-small/ratings.csv') 

# Reader 객체 생성
reader = Reader(rating_scale=(0.5, 5.0))

# 사용자 아이디, 아이템 아이디, 평점 순서 (원래는 timestamp도 있으나 제외)
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# SVD를 이용한 잠재 요인 협업 필터링
algo = SVD(random_state=0)

# 교차 검증 수행
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8739  0.8743  0.8709  0.8729  0.8766  0.8737  0.0018  
MAE (testset)     0.6765  0.6699  0.6671  0.6707  0.6765  0.6721  0.0037  
Fit time          10.79   8.31    8.83    9.03    8.32    9.06    0.91    
Test time         0.30    0.27    0.37    0.25    0.32    0.30    0.04    


{'test_rmse': array([0.87388079, 0.87433591, 0.87091922, 0.87288442, 0.87655467]),
 'test_mae': array([0.67645891, 0.66990717, 0.66713318, 0.67070205, 0.67645393]),
 'fit_time': (10.79212760925293,
  8.314749240875244,
  8.82638430595398,
  9.032832384109497,
  8.32372760772705),
 'test_time': (0.2952089309692383,
  0.270277738571167,
  0.37100720405578613,
  0.2483363151550293,
  0.31914782524108887)}

In [16]:
from surprise.model_selection import GridSearchCV

# n_epochs: SGD 수행 시 반복 횟수, n_factors: 잠재 요인 크기
param_grid = {
    'n_epochs': [20, 40, 60], 
    'n_factors': [50, 100, 200]
}

# GridSearchCV
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3) # algo가 아닌 SVD 입력하였다.
gs.fit(data)

# 최적 하이퍼 파라미터 및 그 때의 최고 성능
print(gs.best_params['rmse'])
print(gs.best_score['rmse'])

{'n_epochs': 20, 'n_factors': 50}
0.8765088141819742


In [17]:
from surprise.dataset import DatasetAutoFolds

# Reader 객체 생성
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))

# DatasetAutoFolds 객체 생성
data_folds = DatasetAutoFolds(ratings_file='./ml-latest-small/ratings_noh.csv', reader=reader)

# 전체 데이터를 train으로 지정
trainset = data_folds.build_full_trainset()

In [18]:
# SVD를 이용한 잠재 요인 협업 필터링 (잠재 요인 크기 = 50)
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [19]:
movies = pd.read_csv('./ml-latest-small/movies.csv')

movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count():
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')
    
print(movies[movies['movieId']==42])

    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [20]:
# 사용자 아이디, 아이템 아이디 문자열로 입력
uid = str(9)
iid = str(42)

# 추천 예측 평점 (.predict)
pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [27]:
# 아직 보지 않은 영화 리스트 함수
def get_unseen_surprise(ratings, movies, userId):
     # 특정 userId가 평점을 매긴 모든 영화 리스트
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()
    
    # 모든 영화명을 list 객체로 만듬. 
    total_movies = movies['movieId'].tolist()
      
    # 한줄 for + if문으로 안 본 영화 리스트 생성
    unseen_movies = [ movie for movie in total_movies if movie not in seen_movies]
    
    # 일부 정보 출력
    total_movie_cnt = len(total_movies)
    seen_cnt = len(seen_movies)
    unseen_cnt = len(unseen_movies)
    
    print(f"평점 매긴 영화 수: {seen_cnt}, 추천 대상 영화 수: {unseen_cnt}, 전체 영화 수: {total_movie_cnt}")
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수: 46, 추천 대상 영화 수: 9696, 전체 영화 수: 9742
